In [2]:
import cv2
import numpy as np
import time
import math
import matplotlib.pyplot as plt
%matplotlib inline

class LivePlot:
    def __init__(self,w = 640,h = 480, yLimit = [0,100],
                interval = 0.001,invert = False,char = " "):
        
        self.yLimit = yLimit
        self.w = w
        self.h = h
        self.invert = invert
        
        # Ara Zaman
        self.interval = interval
        self.char = char[0]
        self.imgPlot = np.zeros((self.h,self.w,3),np.uint8)
        self.imgPlot[:] = 225,225,225
        
        cv2.rectangle(self.imgPlot,(0,0),
                      (self.w,self.h),
                      (0,0,0),cv2.FILLED)
        
        self.xP = 0
        self.yP = 0
        
        self.yList = []
        self.xList = [x for x in range(0,100)]
        
        self.ptime = 0
        
    def update(self,y,color = (255,0,255)):
        
        if time.time() - self.ptime > self.interval:
            
            # White
            self.imgPlot[:] = 225,225,225
            
            self.DrawBackGround()
            
            # Value
            cv2.putText(self.imgPlot,str(y),
                        (self.w - (125),50),cv2.FONT_HERSHEY_PLAIN,3,(150,150,150),3)
            
            if self.invert:
                self.yP = int(np.interp(y,self.yLimit,
                                        [self.h, 0]))                
            else:
                self.yP = int(np.interp(y,self.yLimit,
                                        [0,self.h])) 
                
            self.yList.append(self.yP)
            
            if len(self.yList) == 100:
                self.yList.pop(0)
                
            for i in range(0,len(self.yList)):
                
                if i < 2:
                    pass
                
                else:
                    
                    cv2.line(self.imgPlot, (int((self.xList[i - 1] * (self.w // 100))) - (self.w // 10),
                                            self.yList[i - 1]),
                             (int((self.xList[i] * (self.w // 100)) - (self.w // 10)),
                              self.yList[i]), color, 2)
                    
            self.ptime = time.time()
            
        return self.imgPlot
                     
    def DrawBackGround(self):
        
        # Black Screen !!!
        cv2.rectangle(self.imgPlot,(0,0),
                      (self.w, self.h),
                      (0,0,0),cv2.FILLED)  
        
        # Center Line
        cv2.line(self.imgPlot,(0,self.h // 2), (self.w,self.h // 2),(150,150,150),2)  
        
        # For This Code From up To Down Line
        # w -- > width
        for x in range(0,self.w,50):
            # (0,0),(0,100)
            # (50,0),(50,100)
            # ...
            cv2.line(self.imgPlot,(x,0),(x,self.h),(50,50,50),1)
            
        # For This Code From left To Right Line  
        # h -- > height
        for y in range(0,self.h,50):
            # (0,0),(100,0)
            # (0,50),(100,50)
            # ...
            cv2.line(self.imgPlot,(0,y),(self.w,y),(50,50,50),1)

            cv2.putText(self.imgPlot,
                        f"{int(self.yLimit[1] - ((y / 50)*((self.yLimit[1] - self.yLimit[0]) / (self.h / 50))))}",
                        (10,y),cv2.FONT_HERSHEY_PLAIN,
                        1,(150,150,150),1)
            
        cv2.putText(self.imgPlot,self.char,
                    (self.w - 100, self.h-25),cv2.FONT_HERSHEY_PLAIN,5,(150,150,150),5)
     
        
def main():
    
    xPlot = LivePlot(w=1200, yLimit=[-100, 100], interval=0.01)
    x = 0
    while True:

        x += 1
        if x == 360: 
            x = 0
            
        imgPlot = xPlot.update(int(math.sin(math.radians(x)) * 100))

        cv2.imshow("Image", imgPlot)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

if __name__ == "__main__":
    main()

In [3]:
yLimit = [0,100]
i = [i for i in range(0,480,50)]

def f(i,h = 480):
    #print(((i / 50)*((yLimit[1]-yLimit[0]) / (h / 50)))
    return yLimit[1] - ((i / 50)*((yLimit[1]-yLimit[0]) / (h / 50)))

for j in i:
    print(int(f(j)),end = " ")

100 89 79 68 58 47 37 27 16 6 

In [4]:
import cv2
import mediapipe as mp
import numpy as np
import time
import math

class FaceMeshDetector:
    
    def __init__(self,staticMode = False,maxFaces = 1, landmarks = False,detectionCon = 0.5,trackCon = 0.5):
        
        self.staticMode = staticMode
        self.maxFaces = maxFaces
        self.landmarks = landmarks
        self.detectionCon = detectionCon
        self.trackCon = trackCon
        
        self.mpDraw = mp.solutions.drawing_utils
        self.mpMesh = mp.solutions.face_mesh
        
        self.Mesh = self.mpMesh.FaceMesh(self.staticMode,
                                         self.maxFaces,
                                         self.landmarks,
                                         self.detectionCon,
                                         self.trackCon)
        
        self.drawSpec = self.mpDraw.DrawingSpec(thickness=1,circle_radius=2,color = (0,255,0))
        
        
    def findFaceMesh(self,image,draw = True,id_draw = True):
        
        self.imageRGB = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        self.results = self.Mesh.process(self.imageRGB)
        
        self.faces = []
        h,w,c = image.shape
        
        if self.results.multi_face_landmarks:
            
            for faceLms in self.results.multi_face_landmarks:
                if draw:
                    self.mpDraw.draw_landmarks(image,faceLms,self.mpMesh.FACEMESH_CONTOURS,
                                               self.drawSpec,self.drawSpec)
                    
                face = []
                for id ,lm in enumerate(faceLms.landmark):
                    x,y = int(lm.x*w),int(lm.y*h)
                    face.append([x,y])
                    
                    if id_draw :
                        cv2.putText(image,str(id),(x,y+20),cv2.FONT_HERSHEY_PLAIN,.8,(0,0,255),1)
                    
                self.faces.append(face)
                
        return image,self.faces
    
    def findDistance(self,p1,p2,image= None):
        
        
        x1,y1 = p1
        x2,y2 = p2
        
        cx,cy = (x1+x2) // 2, (y1+y2) //2
        length = math.hypot(x2-x1,y2-y1)
        info = (x1,y1,x2,y2,cx,cy)
        
        if image is not None:
            
            cv2.circle(image,(x1,y1),5,(255,0,255),cv2.FILLED)
            cv2.circle(image,(x2,y2),5,(255,0,255),cv2.FILLED)
            cv2.circle(image,(cx,cy),5,(255,0,255),cv2.FILLED)
            cv2.line(image,(x1,y1),(x2,y2),(255,0,255),2)
            return length,info,image
            
        else:
            
            return length,info
    
    
    def findSelectDraw(self,image,List,color):
        
        imageBlack = np.zeros((image.shape),np.uint8)
        imageColor = image.copy()
        
        cropList = []
        for i in List:
            x,y = self.faces[0][i]
            cropList.append([x,y])
        
        cropList = np.array(cropList)
        
        # İstediğimiz Renk
        imageColor[:] = color
        
        # Arka plan zaten siyah ve biz original resimdeki istediğimiz bölgeyi liste içinde aldık !!!
        # Çıkan sonuç arka taraf siyah istenilen alan beyaz
        imageCrop = cv2.fillPoly(imageBlack,[cropList],(255,255,255))  
        
        # Renk ile siyah beyaz alanı çarptık !!
        mask = cv2.bitwise_and(imageColor,imageCrop)   
        
        # Original resmi çevirdik !!
        imageGray = cv2.cvtColor(image,cv2.COLOR_BGR2GRAY)
        imageGray = cv2.cvtColor(imageGray,cv2.COLOR_GRAY2BGR)
        
        # Ve hangisinin ön planda olmasını istiyorsak belirledik !!!
        results = cv2.addWeighted(imageGray,0.6,mask,0.4,1)
        
        return results

In [32]:
import cv2
import time
import numpy as np


pTime = 0
def fps():
    global pTime
    cTime = time.time()
    fps = 1 / (cTime - pTime)
    pTime = cTime
    return fps


def putTextRect(image,text,pos,scale = 3,thickness = 3 ,colorT = (255,255,255),
                colorR = (255,0,255),font = cv2.FONT_HERSHEY_PLAIN,
                offset = 10,border = None,colorB = (0,255,0)):
    
    ox,oy = pos
    (w,h),_ = cv2.getTextSize(text,font,scale,thickness)
    x1,y1,x2,y2 = ox - offset , oy +offset, ox+w + offset, oy - h - offset 
    
    cv2.rectangle(image,(x1,y1),(x2,y2),colorR,cv2.FILLED)
    
    if border is not None:
        cv2.rectangle(image,(x1,x2),(x2,y2),colorR,cv2.FILLED)
    
    cv2.putText(image,text,(ox,oy),font,scale,colorT,thickness)
    return image,[x1,y2,x2,y1]


cap = cv2.VideoCapture(0)
cap.set(3,640)
cap.set(4,480)


if cap.isOpened():
    pass
else:
    print("Error !!!!")

Mesh_Detector = FaceMeshDetector(maxFaces=1)
plotY = LivePlot(640,480,[20,50],invert = True)

blinkCounter = 0
counter = 0
id_list = [22,23,24,26,110,157,158,159,160,161,130,243]
ratioList = []

color = (255,0,255)

while True:
    
    conn,frame = cap.read()
    frame = cv2.flip(frame,1)
    fps_ = fps()
    cv2.putText(frame,f"FSP : {int(fps_)}",(20,50),cv2.FONT_HERSHEY_PLAIN,2,(255,0,0),3)
    
    frame,faces =Mesh_Detector.findFaceMesh(frame,id_draw = False,draw = False)
    
    if faces:
        #print("Completed !!!!")
        #print(np.array(faces).shape)
        face = faces[0]
        for id in id_list:
            x,y = face[id]
            cv2.circle(frame,(x,y),3,color,cv2.FILLED)
            
        leftUp = face[159]
        leftDown = face[23]
        leftLeft = face[130]
        leftRight = face[243]
        
        lengthVer,_ = Mesh_Detector.findDistance(leftUp,leftDown)
        lengthHor,_ = Mesh_Detector.findDistance(leftLeft,leftRight)
        
        #cv2.putText(frame,f"LengthVer : {int(lengthVer)}",(20,150),cv2.FONT_HERSHEY_PLAIN,3,(255,0,0),3)
        #cv2.putText(frame,f"LengthHor : {int(lengthHor)}",(20,200),cv2.FONT_HERSHEY_PLAIN,3,(255,0,0),3)
        cv2.line(frame,leftUp,leftDown,(0,255,0),2)
        cv2.line(frame,leftLeft,leftRight,(0,255,0),2)
    
        # Doğru orantı kullandık Üçgen örneğini düşün !!! 
        # Misal 2 5 kenarlar var ama 4 10 olsa ikiside doğru orantılı şekilde artmakta
        # Oranları yine 2/5 !!!
        ratio = (lengthVer / lengthHor) * 100
        ratioList.append(ratio)
        
        # Grafikteki yoğunluğu azalttık kısacası !!
        if len(ratioList) > 3:
            ratioList.pop(0)
            
        ratioAvg = sum(ratioList) / len(ratioList)
        
        if ratioAvg < 35 and counter == 0:
            blinkCounter +=1
            counter = 1
            color = (0,255,0)
            
        if counter != 0:
            counter +=1
            if counter > 10:
                counter = 0
                color = (255,0,255)

        
        putTextRect(frame,f"Blink Counter : {blinkCounter}",(20,100))
        imgPlot = plotY.update(ratioAvg,color)
        #cv2.imshow("ImagePlot",imgPlot)
        cv2.imshow("Stack",np.hstack([frame,imgPlot]))
        
    else:
        
        cv2.imshow("Stack",np.hstack([frame,frame]))
         
    #cv2.imshow("Frame",frame)
    
    if cv2.waitKey(1) & 0xff == ord("q"):
        break
        
        
cap.release()
cv2.destroyAllWindows()    